<a href="https://colab.research.google.com/github/muiruriiii/Lux-Tech-Academy-Week-2/blob/main/Week_2_Data_Science_Bootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Let’s say we want to build a model to predict booking prices on Airbnb. Between linear regression and random forest regression, which model would perform better and why?
Dataset: https://www.kaggle.com/code/kalyanmurapaka/airbnb-price-prediction-advanced-regression-method


In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer  # Import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
#Data Collection and Preparation
airbnb = pd.read_csv('/content/train.csv')
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52097 entries, 0 to 52096
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      52097 non-null  int64  
 1   log_price               52097 non-null  float64
 2   property_type           52097 non-null  object 
 3   room_type               52097 non-null  object 
 4   amenities               52097 non-null  object 
 5   accommodates            52097 non-null  int64  
 6   bathrooms               51948 non-null  float64
 7   bed_type                52097 non-null  object 
 8   cancellation_policy     52097 non-null  object 
 9   cleaning_fee            52097 non-null  bool   
 10  city                    52097 non-null  object 
 11  description             52097 non-null  object 
 12  first_review            40908 non-null  object 
 13  host_has_profile_pic    51964 non-null  object 
 14  host_identity_verified  51964 non-null

In [ ]:
airbnb.shape
airbnb.columns

### Linear Regression

In [32]:
#Feature Selection
categorical_features = ['property_type', 'room_type', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'city', 'host_response_rate']
numerical_features = ['accommodates', 'bathrooms', 'number_of_reviews', 'review_scores_rating', 'bedrooms', 'beds']


In [33]:
# Separate categorical and numerical columns
categorical_columns = airbnb.select_dtypes(include=['object']).columns
numerical_columns = airbnb.select_dtypes(include=['number']).columns

In [34]:
# Handle missing values using appropriate strategies for each data type
imputer_categorical = SimpleImputer(strategy='most_frequent')
imputer_numerical = SimpleImputer(strategy='mean')

In [35]:
airbnb[categorical_columns] = imputer_categorical.fit_transform(airbnb[categorical_columns])
airbnb[numerical_columns] = imputer_numerical.fit_transform(airbnb[numerical_columns])

In [36]:
# Create transformers for one-hot encoding categorical features and scaling numerical features
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [37]:
# Combine transformers into a preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [38]:
# Combine preprocessor with a linear regression model into a single pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', LinearRegression())])

In [39]:
# Data Splitting and training the model
X = airbnb.drop('log_price', axis=1)
y = airbnb['log_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
# Model Training
model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['accommodates', 'bathrooms',
                                                   'number_of_reviews',
                                                   'review_scores_rating',
                                                   'bedrooms', 'beds']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['property_type', 'room_type',
                                                   'bed_type',
                                                   'cancellation_policy',
                                                   'cleaning_fee', 'city',
                                                   'host_response_rate'])])),
                ('model', LinearRegression())])

In [41]:
# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


In [42]:
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared (R2): {r2}')

Mean Squared Error: 0.23330864709122678
Root Mean Squared Error: 0.48302033817555423
Mean Absolute Error: 0.36420219211621957
R-squared (R2): 0.5457958507655905


### Random Forest Regression

In [43]:
# Drop Missing Data
airbnb.dropna(inplace=True)


In [44]:
# Feature Selection
# Define the features and target variable
X = airbnb[['accommodates', 'bathrooms', 'bedrooms', 'beds', 'number_of_reviews', 'review_scores_rating']]
y = airbnb['log_price']


In [45]:
# Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [48]:
# Model Selection (Random Forest Regression)
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Model Training
model.fit(X_train, y_train)

# Model Evaluation
# Make predictions on the test set
y_pred = model.predict(X_test)

In [49]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared (R2): {r2}')

Mean Squared Error: 0.32039095368771237
Root Mean Squared Error: 0.5660308769737852
Mean Absolute Error: 0.43418114257502033
R-squared (R2): 0.3762644361602804
